In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_URL'] = os.getenv('OPENAI_BASE_URL')
from langchain.chat_models import ChatOpenAI

In [14]:
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, START, END, MessageGraph
from langchain_core.messages import HumanMessage, SystemMessage,BaseMessage
model=ChatOpenAI(temperature=0,model="glm-4-flash")
# 定义一个消息图
graph = MessageGraph()

# 定义一个节点
graph.add_node("start", model)

# 定义一个从 start 到 END 的边
graph.add_edge("start", END)

# 定义逻辑入口
graph.set_entry_point("start")

# 由图生成一个 runnable
runnable = graph.compile()

res = runnable.invoke(HumanMessage("1加1等于几？"))
print(res)

[HumanMessage(content='1加1等于几？', additional_kwargs={}, response_metadata={}, id='61a62309-768a-42bc-b64e-d4ecdf5b88c0'), AIMessage(content='1加1等于2。这是基本的数学加法运算。', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 11, 'total_tokens': 26, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2d695515-9d73-48ff-93ac-d003a655b393-0')]


In [15]:
from langchain.tools import tool
@tool
def multiply(first_number: int, second_number: int):
    """将2个数相乘"""
    return first_number * second_number

In [16]:
model_with_tools = model.bind(tools=[multiply])

In [17]:
from typing import List
def invoke_model(state: List[BaseMessage]):
    return model_with_tools.invoke(state)

In [18]:
def invoke_tool(state: List[BaseMessage]):
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    multiply_call = None

    # 获得调用参数
    for tool_call in tool_calls:
        if tool_call.get("function").get("name") == "multiply":
            multiply_call = tool_call

    if multiply_call is None:
        raise Exception("No adder input found.")

    # 调用工具：2个数相乘
    res = multiply.invoke(
        json.loads(multiply_call.get("function").get("arguments"))
    )

    # 返回 工具消息
    return ToolMessage(
        tool_call_id=multiply_call.get("id"),
        content=res
    )
    
graph.add_node("multiply", invoke_tool)

graph.add_edge("multiply", END)

Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.


In [ ]:
def router(state: List[BaseMessage]):
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "end"

: 